In [8]:
import pandas as pd
import numpy as np

In [9]:
benign_original = pd.read_csv('DataSet_Original_Repackaged.csv')
malicious_repackaged = pd.read_csv('Repackaged_Malware_Dataset.csv')
benign_repackage = pd.read_csv('DataSet_Benign_Repackaged.csv')

sensor_usage = pd.read_csv('[All]Sensor_Usage_without_authors.csv')

COVID = pd.read_csv('DataSet_COVID_Repackaged_Malware.csv')
COVID_sensor_usage = pd.read_csv('COVID_Sensor_Usage.csv')

In [21]:
COVID

,Package_Name,SHA256,ThumbPrint,Application_Category,Family_Name,Malware_Category,Malware/Benign,Permission: FACTORY_TEST,Permission: DUMP,Permission: BATTERY_STATS,...,Leak: Audio_Video_Eavesdropping,Leak: Location_Lookup,Leak: PIM_Data_Leakage,Leak: Telephony_Identifiers_Leakage,Leak: Device_Settings_Harvesting,Leak: Suspicious_Connection_Establishment,isObfuscated,sdkVersion,targetSdkVersion,label
0,air.com.mteamapps.safiransalamat,6C94071DA2C2510698ED9CE6BD2877F00930014A075F77...,DAD94C32E47CCABF78C53889673AB52877824D67,NaN,NaN,NaN,1,0,0,0,...,0,0,0,1,1,1,0,NaN,NaN,0
1,air.com.mteamapps.safiransalamat,7B9BB74AFEE6AD86D14D6E9B12421A745915CCBC5A09B3...,DAD94C32E47CCABF78C53889673AB52877824D67,NaN,NaN,NaN,1,0,0,0,...,0,0,0,1,1,1,0,NaN,NaN,0
2,co.gov.ins.guardianes,5A50276DB42226325112CBA760139389238402A455F753...,33C7B2B62C57CF154792C11B0E36B19DADBF6DA0,NaN,NaN,NaN,1,0,0,0,...,0,0,0,1,1,0,0,NaN,NaN,0
3,it.softmining.projects.covid19.savelifestyle,02B91E67181FBFA0592BD89040A12A55B99F1DB445C6AD...,C1FFE0970D0BFB74708EEC2A6B0B648C38046815,NaN,NaN,NaN,1,0,0,0,...,0,0,0,0,1,1,0,NaN,NaN,0
4,it.softmining.projects.covid19.savelifestyle,17861CDAF1B3E5CD1888FC9D9DB08173ECE88DCE225F17...,NaN,NaN,NaN,NaN,1,0,0,0,...,0,0,0,0,1,1,0,NaN,NaN,0
5,it.softmining.projects.covid19.savelifestyle,31FDAA0693D4FE16E70909D3F1CA91F9123CD5F874115D...,C1FFE0970D0BFB74708EEC2A6B0B648C38046815,NaN,NaN,NaN,1,0,0,0,...,0,0,0,0,1,0,0,NaN,NaN,0
6,it.softmining.projects.covid19.savelifestyle,3229D5E49632465A7665DD968A960954013666CEBCF9E3...,C1FFE0970D0BFB74708EEC2A6B0B648C38046815,NaN,NaN,NaN,1,0,0,0,...,0,0,0,0,1,1,0,NaN,NaN,0
7,it.softmining.projects.covid19.savelifestyle,47007CE18133CDBC80E07BDF9C8DE9F61E17C741024092...,C1FFE0970D0BFB74708EEC2A6B0B648C38046815,NaN,NaN,NaN,1,0,0,0,...,0,0,0,0,1,1,0,NaN,NaN,0
8,it.softmining.projects.covid19.savelifestyle,75207868EEEDE268B57472560C75FCF5B249C17EAE9058...,C1FFE0970D0BFB74708EEC2A6B0B648C38046815,NaN,NaN,NaN,1,0,0,0,...,0,0,0,0,1,0,0,NaN,NaN,0
9,it.softmining.projects.covid19.savelifestyle,8DA5AACC3AD93C1FC461ACC3FC4D22F02596BDB7E3E6FB...,C1FFE0970D0BFB74708EEC2A6B0B648C38046815,NaN,NaN,NaN,1,0,0,0,...,0,0,0,0,1,1,0,NaN,NaN,0


In [20]:
COVID_sensor_usage

,SHA256,Category,accelerometer,accelerometer_uncalibrated,ambient_temperature,device_private_base,game_rotation_vector,geomagnetic_rotation_vector,gravity,gyroscope,...,pose_6dof,pressure,proximity,relative_humidity,rotation_vector,significant_motion,stationary_detect,step_counter,step_detector,temperature
0,6c94071da2c2510698ed9ce6bd2877f00930014a075f77...,Malware,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7b9bb74afee6ad86d14d6e9b12421a745915ccbc5a09b3...,Malware,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,e6ec7fd477cbebdf081705fa7908b54a2b7d9e552c5bb4...,Malware,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3229d5e49632465a7665dd968a960954013666cebcf9e3...,Malware,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5a50276db42226325112cba760139389238402a455f753...,Malware,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,75207868eeede268b57472560c75fcf5b249c17eae9058...,Malware,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,31fdaa0693d4fe16e70909d3f1ca91f9123cd5f874115d...,Malware,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,47007ce18133cdbc80e07bdf9c8de9f61e17c741024092...,Malware,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,992f9eab66a2846d5c62f7b551e7888d03cea253fa72e3...,Malware,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,8da5aacc3ad93c1fc461acc3fc4d22f02596bdb7e3e6fb...,Malware,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
benign_original.rename(columns = {'MD5': 'SHA256'}, inplace = True)

In [11]:
benign_original['label'] = 0
benign_repackage['label'] = 0

COVID['label'] = 0

In [12]:
malicious_repackaged['label'] = 1

In [13]:
training_dataset = pd.concat([benign_original, malicious_repackaged])

In [14]:
training_dataset = training_dataset.merge(sensor_usage.drop(['Category'], axis = 1), how = 'left', on = 'SHA256').drop_duplicates()
test_dataset = benign_repackage.merge(sensor_usage.drop(['Category'], axis = 1), how = 'left', on = 'SHA256').drop_duplicates()
COVID_test_dataset = COVID.merge(COVID_sensor_usage.drop(['Category'], axis = 1), how = 'left', on = 'SHA256').drop_duplicates()

In [15]:
labels = training_dataset[['label']]
labels_test = test_dataset['label']
labels_COVID_test = COVID_test_dataset[['label']]

training_dataset.drop(['label'], axis = 1, inplace = True)
test_dataset.drop(['label'], axis = 1, inplace = True)
COVID_test_dataset.drop(['label'], axis = 1, inplace = True)

In [18]:
training_dataset.to_csv('Training_Dataset.csv', index = False)
test_dataset.to_csv('Test_Dataset.csv', index = False)
COVID_test_dataset.to_csv('COVID_Test_Dataset.csv', index = False)

In [19]:
labels.to_csv('Labels.csv', index = False)
labels_test.to_csv('Labels_Test.csv', index = False)
labels_COVID_test.to_csv('Labels_COVID_Test.csv', index = False)